In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
from time import sleep
import os

### EDA

In [2]:
train_data = pd.read_csv('training.csv')  
test_data = pd.read_csv('test.csv')
lookid_data = pd.read_csv('IdLookupTable.csv')

In [3]:
train_data.head()

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,...,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...
3,65.225739,37.261774,32.023096,37.261774,60.003339,39.127179,72.314713,38.380967,37.618643,38.754115,...,54.166539,65.598887,72.703722,37.245496,74.195478,50.303165,70.091687,51.561183,78.268383,193 192 193 194 194 194 193 192 168 111 50 12 ...
4,66.725301,39.621261,32.244810,38.042032,58.565890,39.621261,72.515926,39.884466,36.982380,39.094852,...,64.889521,60.671411,77.523239,31.191755,76.997301,44.962748,73.707387,44.227141,86.871166,147 148 160 196 215 214 216 217 219 220 206 18...


### Data Preprocessing

In [9]:
train_data.fillna(method = 'ffill',inplace = True)

In [10]:
def get_pixel_data(data):
    pixel_data = []
    for i in range(len(data)):
        char_list = data[i].split(' ')
        num_list = [0 if x == '' else int(x) for x in char_list]
        pixel_data.append(num_list)
        
    pixel_data = np.array(pixel_data,dtype = 'float')
    pixel_data = pixel_data.reshape(-1,96,96,1)
    return pixel_data
        


In [11]:
#Reshape input data
X_train = get_pixel_data(train_data['Image'])

In [12]:
#Extract location data
labels = train_data.drop('Image',axis = 1)
y_train = [row for row in labels.values]
y_train = np.array(y_train,dtype = 'float')

### Model

In [30]:
import tensorflow as tf
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [14]:
model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['mae'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 32)        288       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 96, 96, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 96, 96, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 32)        9216      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 96, 96, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 32)        0

In [17]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.0, # Randomly zoom image 
        width_shift_range=0.05,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.05,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images


datagen.fit(X_train)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_mae', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size = 0.2)
history = model.fit_generator(datagen.flow(train_x,train_y, batch_size = 100),
                              epochs = 20, validation_data = (valid_x,valid_y),
                              verbose = 2, callbacks=[learning_rate_reduction])

/Users/jixidai/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
57/57 - 231s - loss: 108.2087 - mae: 6.5129 - val_loss: 103.2612 - val_mae: 9.0867
Epoch 2/20
57/57 - 219s - loss: 23.0483 - mae: 3.6576 - val_loss: 57.0566 - val_mae: 6.6310
Epoch 3/20
57/57 - 247s - loss: 20.8129 - mae: 3.4450 - val_loss: 16.4470 - val_mae: 3.2376
Epoch 4/20
57/57 - 226s - loss: 18.3838 - mae: 3.2161 - val_loss: 11.9143 - val_mae: 2.5871
Epoch 5/20
57/57 - 230s - loss: 16.4224 - mae: 3.0071 - val_loss: 8.4868 - val_mae: 2.0245
Epoch 6/20
57/57 - 196s - loss: 15.9052 - mae: 2.9476 - val_loss: 9.3775 - val_mae: 2.1552
Epoch 7/20
57/57 - 217s - loss: 15.9009 - mae: 2.9412 - val_loss: 8.1297 - val_mae: 1.9073
Epoch 8/20
57/57 - 202s - loss: 13.7211 - mae: 2.7008 - val_loss: 6.7838 - val_mae: 1.6401
Epoch 9/20
57/57 - 216s - loss: 13.5877 - mae: 2.6840 - val_loss: 6.6785 - val_mae: 1.6713
Epoch 10/20
57/57 - 229s - loss: 12.7081 - mae: 2.5792 - val_loss: 8.5162 - val_mae: 2.0580
Epoch 11/20
57/57 - 243s - loss: 12.3999 - mae: 2.5484 - val_loss: 6.6465 - val_mae

In [33]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.0, # Randomly zoom image 
        width_shift_range=0.05,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.05,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images
datagen.fit(X_train)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_mae', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size = 0.2)

history = model.fit_generator(datagen.flow(train_x,train_y, batch_size = 100),
                              epochs = 20, validation_data = (valid_x,valid_y),
                              verbose = 2, callbacks=[learning_rate_reduction])

Epoch 1/20
57/57 - 113s - loss: 958.9831 - mae: 22.8345 - val_loss: 543.1647 - val_mae: 20.9671
Epoch 2/20
57/57 - 115s - loss: 16.4057 - mae: 2.9009 - val_loss: 123.0486 - val_mae: 7.8002
Epoch 3/20
57/57 - 116s - loss: 11.3225 - mae: 2.3138 - val_loss: 75.0875 - val_mae: 6.5808
Epoch 4/20
57/57 - 116s - loss: 10.0208 - mae: 2.1362 - val_loss: 37.9296 - val_mae: 4.7962
Epoch 5/20
57/57 - 107s - loss: 9.6602 - mae: 2.0945 - val_loss: 14.3644 - val_mae: 2.7332
Epoch 6/20
57/57 - 115s - loss: 9.0947 - mae: 2.0128 - val_loss: 10.4997 - val_mae: 2.2594
Epoch 7/20
57/57 - 108s - loss: 8.9195 - mae: 1.9979 - val_loss: 8.9756 - val_mae: 2.0096
Epoch 8/20
57/57 - 106s - loss: 8.1381 - mae: 1.8892 - val_loss: 8.5224 - val_mae: 1.9060
Epoch 9/20
57/57 - 118s - loss: 8.0825 - mae: 1.8872 - val_loss: 8.3082 - val_mae: 1.7948
Epoch 10/20
57/57 - 111s - loss: 7.8306 - mae: 1.8594 - val_loss: 6.7888 - val_mae: 1.5336
Epoch 11/20
57/57 - 109s - loss: 7.6910 - mae: 1.8328 - val_loss: 9.6376 - val_mae: 

### Submission

In [18]:
#preparing test data
X_test = get_pixel_data(test_data['Image'])

In [19]:
#DNN with data augmentation public: 3.40
pred = model.predict(X_test)
rowid = list(lookid_data['RowId'])
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])

rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')
loc = loc.clip(0.0,96.0)
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('Submission.csv',index = False)

In [34]:
#Imagenet with data augmentation public: 3.39
pred = model.predict(X_test)
rowid = list(lookid_data['RowId'])
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])

rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')
loc = loc.clip(0.0,96.0)
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('Submission.csv',index = False)